In [1]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

path='imageAttendence/'
images=[]
classNames=[]
mylist=os.listdir(path)
#print(mylist)
for cl in mylist:
    curImg=cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])

print("####IMAGEPROCESSING####")
#print(images)
def findEncoding(images):
    encodeList=[]
    for img in images:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        encode=face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

def markAttendence(name):
    with open('attendence.csv','r+') as f:
        mydatalist=f.readlines()
        nameList = []
        for line in mydatalist:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            date_string = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{date_string}')

encodeListKnown=findEncoding(images)
print('Encoding Completed !!!')
       
cam=cv2.VideoCapture(0)
while True:
    success , img = cam.read()
    imgS=cv2.resize(img,(0,0),None,0.25,0.25)
    imgS=cv2.cvtColor(imgS,cv2.COLOR_BGR2RGB)
    
    FaceCurFrame = face_recognition.face_locations(imgS)
    encodeCurFrame=face_recognition.face_encodings(imgS,FaceCurFrame)
    
    for encodeface , faceloc in zip(encodeCurFrame,FaceCurFrame):
        matches=face_recognition.compare_faces(encodeListKnown,encodeface)
        FaceDis=face_recognition.face_distance(encodeListKnown,encodeface)
        #print(FaceDis)
        
        matchesIndex = np.argmin(FaceDis)
        
        if matches[matchesIndex]:
            name = classNames[matchesIndex].upper()
            print("Face Detected")
            #print(name)
            
            y1,x2,y2,x1=faceloc
            y1,x2,y2,x1=y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            markAttendence(name)
    
    if cv2.waitKey(1)==ord('q'):
        cam.release()
        cv2.destroyAllWindows()
        break
        
        
    cv2.imshow('Webcam',img)
    cv2.waitKey(1)

####IMAGEPROCESSING####
Encoding Completed !!!
Face Detected
Face Detected
Face Detected
Face Detected
Face Detected
Face Detected
Face Detected
Face Detected
Face Detected
Face Detected
Face Detected
Face Detected
Face Detected
Face Detected
